# Merge Tags Workflow

In [ ]:
# Run this workflow to merge tags 
token = "" # @param {type: "string"}

!pip install -q relevanceai==2.7.0
from relevanceai.utils import decode_workflow_token

config = decode_workflow_token(token)

# config = {
#     "inputField": "_surveytag_.B2OE.example.label",
#     "outputField": "_surveytag_.B2OE.example_fixed",
#     "tagsToMerge": {
#         "Teaching": "Teach",
#         "teachers_2": "life"
#     },
#     "authorizationToken": "",
#     "datasetId": "teachers-data-csv"
# }

input_field = config['inputField']
output_field = config['outputField']
tags_to_merge = config['tagsToMerge']

from relevanceai import Client
client = Client(config['authorizationToken'])
ds = client.Dataset(config['dataset_id'])

# Merge the tags together
# Note that the following should work given mutable property of python lists 
# See here to better understand why: https://florimond.dev/en/posts/2018/08/python-mutable-defaults-are-the-source-of-all-evil/
for chunk in ds.chunk_dataset(
    select_fields=[input_field], 
    filters=ds[input_field].exists()
):
  new_chunk = []
  for d in chunk:
    for old_label, new_label in tags_to_merge.items():
      new_doc = {"_id": d['_id']}
      tag_field = ".".join(input_field.split(".")[:-1])
      tag_docs =  client.get_field(tag_field, d)
      tags = client.get_field_across_documents(input_field.split(".")[-1], tag_docs)
      
      new_tags = []
      if old_label in tags:
        if new_label not in new_tags:
            new_tags.append(new_label)
      # To avoid overwriting
      client.set_field(output_field, new_doc, new_tags)
      print(tags)
      print(new_tags)
      new_chunk.append(new_doc)
  # We don't fire and forget here because it would otherwise
  # spam the server
  ds.upsert_documents(new_chunk)